In [3]:
import os
##################  BDD  #####################
DB_SERVER=os.environ.get('DB_SERVER')
USER_DB=os.environ.get('USER_DB')
PASSWORD_DB=os.environ.get('PASSWORD_DB')
DB_NAME=os.environ.get('DB_NAME')

In [29]:
import pandas as pd
import mysql.connector

# Informations de connexion à la base de données MySQL
host = 'db20201-sumertwist.sql-pro.online.net'
database = 'db20201_sumertwist'
user = 'db123270'
password = '123edFrTDE'

# Création du DataFrame à partir de vos données ou d'un fichier CSV par exemple
data = {
    'column1': [1, 2, 3],
    'column2': ['A', 'B', 'C']
}
df = pd.DataFrame(data)

df = pd.read_csv("../raw_data/only_today_export_2024-03-21.csv")

try:
    # Connexion à la base de données MySQL
    connexion = mysql.connector.connect(host=host, database=database, user=user, password=password)

    if connexion.is_connected():
        print('Connexion réussie à la base de données')

        # Création du curseur
        cursor = connexion.cursor()

        compteur=0
        # Insertion des données ligne par ligne
        for index, row in df.iterrows():
            if compteur<1 :
                sql = "INSERT INTO nom_de_votre_table (column1, column2,title) VALUES (%s, %s,%s)"
                cursor.execute(sql, (row['source'], row['theme'],row['title']))
                print(f"Donnée insérée :{compteur}- {row['title']}, {row['theme']}")
                compteur = compteur + 1
                connexion.commit()

        # Validation des changements et fermeture du curseur
        #connexion.commit()
        cursor.close()
        connexion.close()
        print('Données insérées avec succès. Connexion à la base de données fermée')

except mysql.connector.Error as err:
    print("Erreur de connexion à la base de données MySQL :", err)


Connexion réussie à la base de données
Donnée insérée :0- Analyse d’un roman (68e épisode), culture
Données insérées avec succès. Connexion à la base de données fermée


## Test d'une query sur la category

In [4]:
import pandas as pd
import mysql.connector

# Informations de connexion à la base de données MySQL
host = 'db20201-sumertwist.sql-pro.online.net'
database = 'db20201_sumertwist'
user = 'db123270'
password = '123edFrTDE'


# Création de la chaîne de connexion à la base de données
connection_string = f"mysql+mysqlconnector://{user}:{password}@{host}/{database}"

try:
    # Création du moteur SQLAlchemy
    engine = create_engine(connection_string)

    # Requête SQL SELECT
    query = "SELECT * FROM category_dataset"

    # Exécution de la requête et chargement des résultats dans un DataFrame
    cat_df = pd.read_sql_query(query, engine)

except Exception as e:
    print("Erreur lors de l'exécution de la requête :", e)

print("Résultats de la requête :")
cat_df

Erreur lors de l'exécution de la requête : name 'create_engine' is not defined
Résultats de la requête :


NameError: name 'cat_df' is not defined

In [45]:
data_df = pd.read_csv('../raw_data/only_today_export_2024-03-21_cleaned.csv')

# Dataframe which is associate withe the news table
result = data_df.merge(cat_df, left_on='theme', right_on='cat_name')
news_df = result[['category_id', 'title', 'description', 'link','image', 'added_date', 'source', 'soustheme']]
news_df = news_df.rename(columns={'soustheme': 'sub_cat'})
news_df.head(3)

,category_id,title,description,link,image,added_date,source,sub_cat
0,1,Analyse d’un roman (68e épisode),Dans ce nouvel épisode de la série « L’Alchimi...,https://www.aproposdecriture.com/analyse-dun-r...,NaN,2024-03-21 09:00:00,À propos d'écriture,litterature
1,1,Un remake annoncé pour ce film culte des année...,Les producteurs du Discours d'un roi ont annon...,https://www.ecranlarge.com/films/news/1505931-...,NaN,2024-03-21 09:00:00,Écran large,cinema
2,1,Equilibrium : pourquoi ça aurait dû être un gr...,Film de science-fiction sorti au cinéma en 200...,https://www.ecranlarge.com/films/news/1505944-...,NaN,2024-03-21 09:00:00,Écran large,cinema


## IMPORT DES NEWS DANS MYSQL

In [5]:
import pandas as pd
import mysql.connector

# Informations de connexion à la base de données MySQL
host = 'db20201-sumertwist.sql-pro.online.net'
database = 'db20201_sumertwist'
user = 'db123270'
password = '123edFrTDE'

#df = pd.read_csv("../raw_data/only_today_export_2024-03-21.csv")
news_df=news_df.fillna("")

try:
    # Connexion à la base de données MySQL
    connexion = mysql.connector.connect(host=host, database=database, user=user, password=password)

    if connexion.is_connected():
        print('Connexion réussie à la base de données')

        # Création du curseur
        cursor = connexion.cursor()

        compteur=0
        # Insertion des données ligne par ligne
        for index, row in news_df.iterrows():
            if compteur<1000000 :
                sql = "INSERT INTO news_dataset (category_id,title,description,link,image,added_date,source,sub_cat) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
                cursor.execute(sql, (row['category_id'], row['title'],row['description'],row['link'],row['image'],row['added_date'],row['source'],row['sub_cat']))
                print(f"Donnée insérée :{compteur}- {row['title']}, {row['category_id']}")
                compteur = compteur + 1
                #connexion.commit()

        # Validation des changements et fermeture du curseur
        connexion.commit()
        cursor.close()
        connexion.close()
        print('Données insérées avec succès. Connexion à la base de données fermée')

except mysql.connector.Error as err:
    print("Erreur de connexion à la base de données MySQL :", err)


NameError: name 'news_df' is not defined

In [2]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql

# initialize Connector object
connector = Connector()

# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(
        DB_SERVER,
        "pymysql",
        user=USER_DB,
        password=PASSWORD_DB,
        db=DB_NAME
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)
pool.connect()


In [ ]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql

data_df = pd.read_csv('../raw_data/only_today_export_2024-03-21_cleaned.csv')

# Dataframe which is associate withe the news table
result = data_df.merge(cat_df, left_on='theme', right_on='cat_name')
news_df = result[['category_id', 'title', 'description', 'link','image', 'added_date', 'source', 'soustheme']]
news_df = news_df.rename(columns={'soustheme': 'sub_cat'})
news_df.head(3)
news_df=news_df.fillna("")


# initialize Connector object
connector = Connector()

# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(
        DB_SERVER,
        "pymysql",
        user=USER_DB,
        password=PASSWORD_DB,
        db=DB_NAME
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)
pool.connect()


for index, row in news_df.iterrows():
            if compteur<1000000 :
                sql = "INSERT INTO news_dataset (category_id,title,description,link,image,added_date,source,sub_cat) VALUES (%s,%s,%s,%s,%s,%s,%s,%s)"
                cursor.execute(sql, (row['category_id'], row['title'],row['description'],row['link'],row['image'],row['added_date'],row['source'],row['sub_cat']))
                print(f"Donnée insérée :{compteur}- {row['title']}, {row['category_id']}")
                compteur = compteur + 1
                #connexion.commit()


In [6]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql
import pandas as pd

# initialize Connector object
connector = Connector()

# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(
        DB_SERVER,
        "pymysql",
        user=USER_DB,
        password=PASSWORD_DB,
        db=DB_NAME
    )
    return conn

# create connection pool
pool = sqlalchemy.create_engine(
    "mysql+pymysql://",
    creator=getconn,
)
pool.connect()

# Requête SQL SELECT
query = "SELECT * FROM category_dataset"

# Exécution de la requête et chargement des résultats dans un DataFrame
df = pd.read_sql(query, pool)

print("Résultats de la requête :")
df

Résultats de la requête :


,category_id,cat_name
0,1,culture
1,2,musique
2,3,technologie
3,4,politique
4,5,sport
5,6,economie
6,7,environnement
7,8,general
8,9,emploi
9,10,cite


In [6]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql
import pandas as pd

# initialize Connector object
connector = Connector()

# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(DB_SERVER,"pymysql",user=USER_DB,password=PASSWORD_DB,
                                                             db=DB_NAME)
    return conn

# create connection pool
pool = sqlalchemy.create_engine("mysql+pymysql://",creator=getconn)
pool.connect()

# Requête SQL SELECT
query = "SELECT * FROM category_dataset"

# Exécution de la requête et chargement des résultats dans un DataFrame
cat_df = pd.read_sql(query, pool)

print("Résultats de la requête :")
cat_df

Résultats de la requête :


,category_id,cat_name
0,3,technologie
1,4,politique
2,5,sport
3,6,economie
4,8,general
5,9,emploi
6,11,management
7,12,presse


## INSERTION REVIEW

In [ ]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql
import pandas as pd

# initialize Connector object
connector = Connector()

# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(DB_SERVER,"pymysql",user=USER_DB,password=PASSWORD_DB,
                                                             db=DB_NAME)
    return conn

# create connection pool
pool = sqlalchemy.create_engine("mysql+pymysql://",creator=getconn)


with pool.connect() as db_conn:
    # Requête SQL SELECT
    # insert data into our ratings table
    insert_stmt = sqlalchemy.text(
        "INSERT INTO review_dataset (user_id, news_id, like_the_news,good_recommendation,updated_date) VALUES (:user_id, :news_id, :like_the_news,:good_recommendation,:updated_date)",
    )

    db_conn.execute(insert_stmt, parameters={"user_id": 1, "news_id": 123, "like_the_news": 1,"good_recommendation":0,"updated_date":'2024-03-21 12:34:56'})
    # commit transactions
    db_conn.commit()


['the-mdr-project:europe-west1:mdr-project']: An error occurred while performing refresh. Scheduling another refresh attempt immediately
Traceback (most recent call last):
  File "/Users/denislouvegnies/.pyenv/versions/3.10.6/envs/the_mdr_project/lib/python3.10/site-packages/google/cloud/sql/connector/instance.py", line 312, in _refresh_task
    refresh_data = await refresh_task
  File "/Users/denislouvegnies/.pyenv/versions/3.10.6/envs/the_mdr_project/lib/python3.10/site-packages/google/cloud/sql/connector/instance.py", line 260, in _perform_refresh
    ephemeral_cert, expiration = await ephemeral_task
  File "/Users/denislouvegnies/.pyenv/versions/3.10.6/envs/the_mdr_project/lib/python3.10/site-packages/google/cloud/sql/connector/client.py", line 193, in _get_ephemeral
    resp = await self._client.post(
  File "/Users/denislouvegnies/.pyenv/versions/3.10.6/envs/the_mdr_project/lib/python3.10/site-packages/aiohttp/client.py", line 693, in _request
    resp.raise_for_status()
  File "

In [31]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql
import pandas as pd

# initialize Connector object
connector = Connector()
# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(DB_SERVER,"pymysql",user=USER_DB,password=PASSWORD_DB,db=DB_NAME)
    return conn
# create connection pool
pool = sqlalchemy.create_engine("mysql+pymysql://",creator=getconn)


with pool.connect() as db_conn:
    # Requête SQL SELECT
    # insert data into our ratings table
    insert_stmt = sqlalchemy.text(
        "INSERT INTO review_dataset (user_id, news_id, like_the_news,updated_date) VALUES (:user_id, :news_id, :like_the_news,:updated_date)",
    )

    db_conn.execute(insert_stmt, parameters={"user_id": 2, "news_id": 4545, "like_the_news": 1,"updated_date":'2024-03-21 16:34:56'})
    # commit transactions
    rep=db_conn.commit()
    print(rep)


None


## INSERTION DES NEWS DU JOUR 7AM

In [7]:
import pandas as pd

data_df = pd.read_csv('../raw_data/only_today_export_2024-03-28_cleaned.csv')

# Dataframe which is associate withe the news table
result = data_df.merge(cat_df, left_on='theme', right_on='cat_name')
news_df = result[['category_id', 'title', 'description', 'link','image', 'added_date', 'source', 'soustheme']]
news_df = news_df.rename(columns={'soustheme': 'sub_cat'})
news_df.head(3)
news_df=news_df.fillna("")

In [8]:
news_df.shape

(7802, 8)

In [9]:
news_df.head(5)

,category_id,title,description,link,image,added_date,source,sub_cat
0,3,"Ateliers « pitch » des élèves de 1re année, av...","Vendredi 15 mars, atelier pitch, groupe 9, ave...",http://blog.enssat.fr/2024/03/ateliers-pitch-d...,,2024-03-28 09:00:00,École Nationale Supérieure des Sciences Appliq...,
1,3,"L’outil (en) didactique : quoi, pour quoi, com...",En tant qu’aides au service des apprentissages...,https://eduveille.hypotheses.org/18970,,2024-03-28 09:00:00,Éduveille,informatique
2,3,Collectiviser ensemble les épreuves des profes...,Lætitia Progin est docteure en sciences de l&#...,https://eduveille.hypotheses.org/18731,,2024-03-28 09:00:00,Éduveille,informatique
3,3,Test de la Withings ScanWatch Nova : plus fine...,Avec son design affiné et son nouveau capteur ...,https://www.01net.com/tests/withings-scanwatch...,,2024-03-28 09:00:00,01net : tests,informatique
4,3,Apple peut mettre à jour les iPhone sans les s...,"À l'achat d'un nouveau smartphone, la seule en...",https://www.01net.com/actualites/apple-peut-me...,,2024-03-28 09:00:00,01net : toute l'actualité,informatique


In [10]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql

#LE news_df EST FAIT AU DESSUS

#https://colab.research.google.com/github/GoogleCloudPlatform/cloud-sql-python-connector/blob/main/samples/notebooks/mysql_python_connector.ipynb#scrollTo=D3xMRsBl3Ihl


# initialize Connector object
connector = Connector()
# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(DB_SERVER,"pymysql",user=USER_DB,password=PASSWORD_DB,db=DB_NAME,charset='utf8mb4',collation='utf8mb4_unicode_ci')
    return conn
# create connection pool
pool = sqlalchemy.create_engine("mysql+pymysql://",creator=getconn)


pool.connect()
compteur=0
with pool.connect() as db_conn:
    for index, row in news_df.iterrows():
        if compteur<20000 :
            try :
                insert_stmt = sqlalchemy.text(
                    "INSERT INTO news_dataset (category_id,title,description,link,image,added_date,source,sub_cat) VALUES (:category_id,:title,:description,:link,:image,:added_date,:source,:sub_cat)",
                )
                db_conn.execute(insert_stmt, parameters={"category_id": row['category_id'], "title": row['title'], "description": row['description'],"link":row['link'],"image":row['image'],"added_date":row['added_date'],"source":row['source'],"sub_cat":row['sub_cat']})
                print(f"Donnée insérée :{compteur}- {row['title']}, {row['category_id']}")
            except :
                print(f"ERROR --- row['title']")
            compteur = compteur + 1

    rep=db_conn.commit()
print("Done !!!")

Donnée insérée :0- Ateliers « pitch » des élèves de 1re année, avec l'appui de Nokia et Orange, 3
Donnée insérée :1- L’outil (en) didactique : quoi, pour quoi, comment ?, 3
Donnée insérée :2- Collectiviser ensemble les épreuves des professionnel.les de l’établissement scolaire : un enjeu de recherche et de formation, 3
Donnée insérée :3- Test de la Withings ScanWatch Nova : plus fine et légère, vraiment premium ?, 3
Donnée insérée :4- Apple peut mettre à jour les iPhone sans les sortir de leur boîte, 3
Donnée insérée :5- Google va alléger l’interface de Maps et ajouter des recommandations de visite, 3
Donnée insérée :6- 01net morning : un Pixel 9 Pro XL dans le chapeau de Google, hack de France Travail et test de la nouvelle TV OLED de Samsung, 3
Donnée insérée :7- Possible erreur, l’ultra haut de gamme OnePlus 12 est à -51% jusqu’à ce soir minuit ⚡, 3
Donnée insérée :8- MacBook Pro : Amazon ridiculise Apple avec un prix FOU sur le modèle avec puce M3 Pro, 3
Donnée insérée :9- Amazon s

## Récupération du DF NEWS

In [46]:
from google.cloud.sql.connector import Connector
import sqlalchemy
import pymysql
import pandas as pd

# initialize Connector object
connector = Connector()

# function to return the database connection
def getconn() -> pymysql.connections.Connection:
    conn: pymysql.connections.Connection = connector.connect(DB_SERVER,"pymysql",user=USER_DB,password=PASSWORD_DB,
                                                             db=DB_NAME)
    return conn

# create connection pool
pool = sqlalchemy.create_engine("mysql+pymysql://",creator=getconn)
pool.connect()

# Requête SQL SELECT
query = "SELECT * FROM news_dataset where added_date ='2024-03-21 09:00:00'"

# Exécution de la requête et chargement des résultats dans un DataFrame
news_df = pd.read_sql(query, pool)

print("Résultats de la requête :")
news_df

Résultats de la requête :


,news_id,category_id,title,description,link,image,added_date,source,sub_cat
0,1,1,Analyse d’un roman (68e épisode),Dans ce nouvel épisode de la série « L’Alchimi...,https://www.aproposdecriture.com/analyse-dun-r...,,2024-03-21 09:00:00,À propos d'écriture,litterature
1,2,1,Un remake annoncé pour ce film culte des année...,Les producteurs du Discours d'un roi ont annon...,https://www.ecranlarge.com/films/news/1505931-...,,2024-03-21 09:00:00,Écran large,cinema
2,3,1,Equilibrium : pourquoi ça aurait dû être un gr...,Film de science-fiction sorti au cinéma en 200...,https://www.ecranlarge.com/films/news/1505944-...,,2024-03-21 09:00:00,Écran large,cinema
3,4,1,Furiosa : George Miller explique pourquoi il a...,Le réalisateur George Miller est revenu sur la...,https://www.ecranlarge.com/films/news/1505918-...,,2024-03-21 09:00:00,Écran large,cinema
4,5,1,Ce vidéoprojecteur 4K LG profite d'une remise ...,Reconnu comme l'un des meilleurs modèles à ult...,https://www.ecranlarge.com/high-tech/news/1505...,,2024-03-21 09:00:00,Écran large,cinema
...,...,...,...,...,...,...,...,...,...
12880,12881,12,"Immobilier : Paris, Cannes, Aix-en-Provence, L...","Mardi 19 mars 2024, le site d’estimation en li...",https://www.femmeactuelle.fr/actu/news-actu/im...,https://fac.img.pmdstatic.net/fit/~1~fac~2024~...,2024-03-21 09:00:00,Femme actuelle,feminine
12881,12882,12,"""Nous t’aimons"" : Bruce Willis célèbre son ann...","Mardi 19 mars 2024, Bruce Willis a célébré ses...",https://www.femmeactuelle.fr/actu/news-actu/no...,https://fac.img.pmdstatic.net/fit/~1~fac~2024~...,2024-03-21 09:00:00,Femme actuelle,feminine
12882,12883,12,Kate Middleton remplacée par une doublure ? He...,Un sosie de Kate Middleton a démenti s’être fa...,https://www.femmeactuelle.fr/actu/news-actu/ka...,https://fac.img.pmdstatic.net/fit/~1~fac~2024~...,2024-03-21 09:00:00,Femme actuelle,feminine
12883,12884,12,Mort de la journaliste politique Laëtitia Krup...,"Mercredi 20 mars 2024, la société de productio...",https://www.femmeactuelle.fr/actu/news-actu/mo...,https://fac.img.pmdstatic.net/fit/~1~fac~2024~...,2024-03-21 09:00:00,Femme actuelle,feminine
